In [1]:
import pymongo
import pandas as pd
import numpy as np
from numpy.linalg import inv
from scipy import signal
from scipy.io import wavfile
import soundfile as sf
import matplotlib.pyplot as plt
from bson.objectid import ObjectId
import json
import pprint
from pydub import AudioSegment
import urllib
import fnmatch
import os
import requests
import ffmpy
import subprocess
import librosa
import librosa.display

%matplotlib inline

In [2]:
client = pymongo.MongoClient('localhost',27017)
#client = pymongo.MongoClient('localhost',32768)
db = client.get_database('b=bandhub')
songCol = db.get_collection('songsStream')
vidCol = db.get_collection('mergedVideos')
trackCol = db.get_collection('tracksStream')
postCol = db.get_collection('posts')
[songCol.count(), vidCol.count(), trackCol.count(), postCol.count()]

[425706, 198169, 915582, 494867]

In [32]:
def get_published_tracks(num_track):
    num_track_mixes = []
    master_song_ids = []
    pub_tracks = postCol.find({'songAccess': 1}, {'participantsInfo':1 , 'objectId':1})
    for docs in pub_tracks:
        #print docs['_id']
        if(len(docs['participantsInfo']['publishedTracks']) == num_track):
            yield docs
            #ThreeTrackMixes.append(docs['objectId'])
            #num_track_mixes.append(docs['_id'])
            #master_song_ids.append(docs['objectId'])
    #count_track_mixes = len(num_track_mixes)
    #return num_track_mixes[idx],master_song_ids[idx],count_track_mixes

In [33]:
#doc = [x for x in get_published_tracks(3)]
# print x
# doc = pub_track_generator
print len(doc)

3908


In [109]:
def get_filename(file):
    filePath = file.split('/')
    return filePath[- 1]

In [145]:
def get_processed_audio_ids(tracks_per_mix):
    docs = [x for x in get_published_tracks(tracks_per_mix)]
    count_mixes = len(docs)
    missing_song_id = []
    pub_songs = []
    for j in range(450,600):
    #for j in range(count_mixes):
        remix_test = docs[j]['_id']
        song_id = docs[j]['objectId']
        #remix_test,song_id,count_track_mixes = get_published_tracks(tracks_per_mix, j)
        pub_tracks = []
        if docs[j]['_id'] == remix_test:
            #for docs in rmt1:
            for i in range(tracks_per_mix):
                pub_tracks.append(docs[j]['participantsInfo']['publishedTracks'][i]['_id'])

        songLookUp = songCol.find({'masterSongId' : song_id})
        if songLookUp is -1:
            missing_song_id.append(song_id)
            pprint.pprint('Error: SongID not found in Song Collection!')
        else:
            for songdocs in songLookUp:
                for i in range(tracks_per_mix):
                    if(str(pub_tracks[i]) not in songdocs['settings']):
                        continue
                    elif ('effectsAudioUrl' in songdocs['settings'][str(pub_tracks[i])]): 
                        gettrack = trackCol.find({'_id' : pub_tracks[i]})
                        for tdocs in gettrack:
                            if tdocs['startTimeValue'] is not 0:
                                pub_songs.append([remix_test,song_id])
                            continue
                            
                    else:
                        continue
    pprint.pprint(pub_songs)
    return pub_songs
    #pprint.pprint(indexRequired)

In [122]:
def get_mixing_coeff(req_idx):
    #track_id, song_id, 
    #idx = indexRequired[req_idx]
    NumTrack = 3
    proc_id = get_processed_audio_ids(NumTrack)
    track_id = proc_id[req_idx][0]
    song_id = proc_id[req_idx][1]
    
    pub_tracks = []
    pub_t_vol = []
    ogg_paths = []
    start_times = []
    track_filenames = []
    songDoc = songCol.find({'masterSongId' : song_id})
    rmtDoc = postCol.find({'_id': track_id})
    for docs in songDoc:
        #pprint.pprint(docs)
        #pprint.pprint('============')
        for rdocs in rmtDoc:
            for i in range(NumTrack):
                pub_tracks.append(rdocs['participantsInfo']['publishedTracks'][i]['_id'])
        #pprint.pprint(len(pub_tracks))
        for i in range(NumTrack):
            pub_t_vol.append(docs['settings'][str(pub_tracks[i])]['volume'])
            if('effectsAudioUrl' in docs['settings'][str(pub_tracks[i])]): 
                ogg_paths.append(docs['settings'][str(pub_tracks[i])]['effectsAudioUrl'])
                gettrack = trackCol.find({'_id' : pub_tracks[i]})
                for tdocs in gettrack:
                    start_times.append(tdocs['startTimeValue'])
            else:
                gettrack = trackCol.find({'_id' : pub_tracks[i]})
                for tdocs in gettrack:
                    #pprint.pprint(tdocs)
                    ogg_paths.append(tdocs['audioChannels'][0]['fileUrl'])
                    start_times.append(tdocs['startTimeValue'])
    
    return pub_t_vol, ogg_paths, start_times

In [116]:
def download_track(ogg_path):
    
    path = '/scratch/rrs432/tracks'
    #os.mkdir(path)
    os.chdir(path)
    

    r = requests.get(ogg_path)
    filename = get_filename(ogg_path)
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)
            
    print('Track download successful!')

In [130]:
def download_mix_video(song_id):
    pointer = vidCol.find({'songId': song_id})
    for songdocs in pointer:
        premix = songdocs['oggMergedVideoUrl']
        
    download_track(premix)

In [149]:
s1 = '549a29940fa0cc4515916469'
s = ObjectId(s1)
download_mix_video(s)

Track download successful!


In [138]:
def pad_audio(filenames):
    
    oggTracks = []
    trackLens = []
    samRates = []
    
    for i in range(len(filenames)):
        
        oggTrack, samRate = sf.read('/scratch/rrs432/tracks/'+filenames[i])
        samRates.append(samRate)
        trackLens.append(oggTrack.shape[0])
        
        #Convert mono to stereo
        if oggTrack.ndim is not 2:
            oggTrack = np.stack((oggTrack, oggTrack), axis = 1)
        oggTracks.append(oggTrack)

        pprint.pprint(np.shape(oggTracks[i]))
        
    if (min(samRates) == max(samRates)):
        mixsr = samRates[0] 
    else:
        pprint.pprint('Warning: Sample rates unequal')
    
    masterLen = max(trackLens)
    
    for i in range(len(filenames)):
        diff = np.zeros(((masterLen - trackLens[i]),2))
        oggTracks[i] = np.vstack((oggTracks[i], diff))
        pprint.pprint(np.shape(oggTracks[i]))
        
    return oggTracks

In [146]:
a, b, c = get_mixing_coeff(1)

[[ObjectId('549a3182b76485d289844440'), ObjectId('549a29940fa0cc4515916469')],
 [ObjectId('549a3182b76485d289844440'), ObjectId('549a29940fa0cc4515916469')],
 [ObjectId('54c0dd70b76485d289898b46'), ObjectId('54c0b083c7d27f5d7f9b0edf')]]


In [131]:
pprint.pprint(b)

[u'http://bandhubwebmedia3.blob.core.windows.net/files/bee57119-d6b9-4657-58c4-9c5a1a67abca.ogg',
 u'http://bandhubwebmedia0.blob.core.windows.net/files/70523227-c979-438f-8d5e-b2e641716060.ogg',
 u'http://bandhubwebmedia9.blob.core.windows.net/files/469df6e0-2c08-4be1-9bf3-7ffdd2d22242.ogg']


In [141]:
download_track(b[0])
download_track(b[1])
download_track(b[2])

Track download successful!
Track download successful!
Track download successful!


In [142]:
!ls

469df6e0-2c08-4be1-9bf3-7ffdd2d22242.ogg
70523227-c979-438f-8d5e-b2e641716060.ogg
bee57119-d6b9-4657-58c4-9c5a1a67abca.ogg


In [143]:
b1 = []
for i in range(0,3):
    b1.append(get_filename(b[i]))
#pprint.pprint(b1)
pad_audio(b1)

(9007104, 2)
(9173248, 2)
(9173248, 2)
[9007104, 9173248, 9173248]
9173248
(9173248, 2)
(9173248, 2)
(9173248, 2)


In [151]:
def rename_file(file, filename):
    for file in os.listdir('.'):
        if fnmatch.fnmatch(file, 'f-*'):
            rename_cmd = ['mv', file, filename]
            s = subprocess.call(rename_cmd)
            if s is 0:
                print('success')

In [152]:
rename_file(s1,'premix.ogg')

success


In [153]:
!ls

469df6e0-2c08-4be1-9bf3-7ffdd2d22242.ogg
70523227-c979-438f-8d5e-b2e641716060.ogg
bee57119-d6b9-4657-58c4-9c5a1a67abca.ogg
premix.ogg


In [155]:
def strip_audio_from_video():
    convert_cmd = ['ffmpeg', '-i', 'premix.ogg', '-vsync', '1', '-async', '1', '-vn', 'premix.wav']
    subprocess.call(convert_cmd)

In [157]:
def compute_estimate_mix(track_filenames):
    
    ogg_tracks = pad_audio(track_filenames)
    mixOgg = 0
    for i in range(len(track_filenames)):
        mixOgg += (pub_t_vol[i]*np.array(ogg_tracks[i]))
    sf.write('estimix.wav', mixOgg / np.max(np.abs(mixOgg)), mixsr)
    print('Estimate mix computation succesful!')